# KQCircuits PCell Viewer

Lightweight option for viewing/testing pcells under development.

## Configuration

In [2]:
# ------------------------------------------------------------
# IMPORTS
# ------------------------------------------------------------
import sys
import json
import time
import shutil
import logging
from pathlib import Path
from importlib import import_module
from autologging import TRACE

# Python library for handling GDSII files
import gdspy
# Notebook display tools
from IPython.display import display, Image, SVG
# KLayout standalone Python library  
import klayout.db as pya

# KQCircuits helper modules   
import kqcircuits.util.library_helper as library_helper

logging.basicConfig(
    level=logging.DEBUG, 
    stream=sys.stdout,
    format="%(asctime)s:%(levelname)s:%(name)s:%(funcName)s:%(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# ------------------------------------------------------------
# INPUT
# ------------------------------------------------------------
class_name = "Chip"
module_name = library_helper.to_module_name(class_name)
# Override default parameters as needed
parameters = {}
# View options 
use_timestamp = False
scale = 0.05
show_gds = False
show_svg = True

# ------------------------------------------------------------
# VIEW CONFIGURATION
# ------------------------------------------------------------
library_name = library_helper.to_library_name(class_name)

# import module
try:
    module = import_module("kqcircuits.elements." + module_name)
except ImportError:
    try:
        module = import_module("kqcircuits.chips." + module_name)
    except ImportError:
        try:
            module = import_module("kqcircuits.test_structures." + module_name)
        except ImportError as e:
            raise Exception("Failed to find module {}.".format(module_name)) from e

# get class
cls = getattr(module, class_name)

# get default parameters
params = cls().get_parameters()
defaults = {}
for p in params:
    defaults[p.name] = p.default if isinstance(p.default, (str, int, float, bool)) else str(p.default)

# print configuration
print("MODULE NAME: {}".format(module_name))
print("CLASS NAME: {}".format(class_name))
print("LIBRARY NAME: {}".format(library_name))
print()
print("PARAMETERS")
print("----------")
print("DEFAULT: {}".format(json.dumps(defaults, indent=2)))
print("INPUT: {}".format(json.dumps(parameters, indent=2)))
print()
print("OPTIONS")
print("----------")
print("USE TIMESTAMP: {}".format(use_timestamp))
print("SCALE: {}".format(scale))
print("SHOW GDS: {}".format(show_gds))
print("SHOW SVG: {}".format(show_svg))


## Images

In [3]:
# ------------------------------------------------------------
# IMAGE VARIABLES
# ------------------------------------------------------------

# Derive image directory name
if use_timestamp:
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    dir_name = timestamp + "_" + module_name + "_images"
else:
    dir_name = module_name + "_images"

# Create temporary image directory
dir_path = Path.cwd().parent.joinpath("tmp").joinpath(dir_name)
if dir_path.exists():
    if dir_path.is_dir():
        shutil.rmtree(dir_path)
dir_path.mkdir()

# Define paths for storing image files
gds_path = dir_path.joinpath(module_name + ".gds")
svg_path = dir_path.joinpath(module_name + ".svg")

# ------------------------------------------------------------
# CREATE & EXPORT LAYOUT
# ------------------------------------------------------------
layout = pya.Layout()
top = layout.create_cell("top")
cell = cls.create(layout, **parameters)

top.insert(pya.DCellInstArray(cell.cell_index(), pya.DTrans()))

# Export to gds
layout.write(str(gds_path))

# Export to svg
if show_svg:
    gdspy_library = gdspy.GdsLibrary(infile=str(gds_path))
    gdspy_library.cells[library_name.replace(" ", "$")].write_svg(str(svg_path), scaling=scale)

# ------------------------------------------------------------
# VIEW SVG
# ------------------------------------------------------------
if show_svg:
    svg_image = SVG(str(svg_path))
    display(svg_image)

# ------------------------------------------------------------
# VIEW GDS
# ------------------------------------------------------------
if show_gds: gdspy.LayoutViewer()

# clean up gdspy library
gdspy.current_library = gdspy.GdsLibrary()